In [ ]:
import re  # Importing the regular expressions library.
import plotly.graph_objects as go  # Importing Plotly for interactive data visualization.
import matplotlib.pyplot as plt  # Importing Matplotlib for static data visualization.

# Define patterns for different types of PII information along with weights.
patterns = {
    "Email": {"pattern": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", "weight": 1},
    "Phone Number": {"pattern": r"\b(?:\+?1[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b", "weight": 2},
    "Social Security Number": {"pattern": r"\b\d{3}[-.\s]?\d{2}[-.\s]?\d{4}\b", "weight": 0},
    "Name": {"pattern": r"\b[A-Z][a-z]+\s[A-Z][a-z]+\b", "weight": 1}
}

# Function to find PII information in text using patterns.
def detect_pii(text):
    detected_pii = {}
    for key, pattern_info in patterns.items():
        matches = re.findall(pattern_info["pattern"], text)
        if matches:
            detected_pii[key] = matches
    return detected_pii

# Function to display the PII score and Bar chart.
def display_results(scanned_text, detected_pii, pii_score):
    # Calculate the count of Detected and Total PII occurrences.
    detected_pii_counts = {key: len(matches) for key, matches in detected_pii.items()}
    total_pii_counts = {key: len(re.findall(pattern_info["pattern"], scanned_text)) for key, pattern_info in patterns.items()}
    
    # Create a Bar chart to visualize Detected PII vs Total PII.
    plt.figure(figsize=(10, 6))
    bar_width = 0.4
    index = range(len(detected_pii_counts))
    plt.bar(index, detected_pii_counts.values(), bar_width, color='black', label='Detected PII')
    plt.bar([i + bar_width for i in index], total_pii_counts.values(), bar_width, color='teal', alpha=0.7, label='Total PII in Text')
    plt.xlabel('PII Type')
    plt.ylabel('Count')
    plt.title('Detected PII vs Total PII')
    plt.xticks([i + bar_width / 2 for i in index], detected_pii_counts.keys(), rotation=90, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Create a gauge indicator to display the PII score.
    fig = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=pii_score,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "PII Score Indicator", 'font': {'size': 24}},
        delta={'reference': 20, 'increasing': {'color': "RebeccaPurple"}},
        gauge={
            'axis': {'range': [None, 500], 'tickwidth': 1, 'tickcolor': "darkblue"},
            'bar': {'color': "darkblue"},
            'bgcolor': "white",
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 250], 'color': 'cyan'},
                {'range': [250, 400], 'color': 'royalblue'}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 490}}))
    fig.update_layout(paper_bgcolor="lavender", font={'color': "darkblue", 'family': "Arial"})
    fig.show()

# Function to read text from a file.
def scan_system_for_pii(file_path):
    with open(file_path, "r") as file:
        text_to_scan = file.read()
    return text_to_scan

# Ask user for consent before scanning for PII.
user_consent = input("Do you agree to a PII scan of your system? (yes/no): ")
if user_consent.lower() == "yes":
    file_path = "PII_Data.txt"  # Define the file path.
    scanned_text = scan_system_for_pii(file_path)  # Read text from the file.
    detected_pii = detect_pii(scanned_text)  # Detect PII in the text.
    
    # Calculate the PII score by considering the weighted counts.
    pii_score = sum(len(matches) * pattern_info["weight"] for key, matches in detected_pii.items() for pattern, pattern_info in patterns.items() if pattern == key)
    
    # Display the results.
    display_results(scanned_text, detected_pii, pii_score)
    
    # Provide recommendations based on the PII score.
    if pii_score > 20:
        print("Your weighted PII score is high. Consider taking actions to improve system security.")
        print("For solutions, you can visit Galaxkey Enterprises: https://www.galaxkey.com/")
    else:
        print("Your weighted PII score is low. You can still take steps to enhance system security.")
        print("For solutions, you can visit Galaxkey Enterprises: https://www.galaxkey.com/")
else:
    print("PII scanning requires user consent.")